In [227]:
# EXAMPLE gym different environments
import gym
import numpy as np
import time
#import time
#print(gym.__version__)

#env = gym.make('LunarLander')
#env = gym.make('CartPole-v0')
env = gym.make('FrozenLake-v1')
observation = env.reset(seed=42)
action_size = env.action_space.n
print('Number of actions:', action_size)

done = False
n_episodes = 1
for _ in range(n_episodes):
    while not done:
        action = np.random.randint(0, action_size)
        observation, reward, done, info,_ = env.step(action)
        print('Action:', action)
        print('Observation:', observation)
        print('Reward:', reward)
        print(info,done)
        #env.render()
    done = False
    env.reset()
env.close()


Number of actions: 4
Action: 1
Observation: 4
Reward: 0.0
False False
Action: 3
Observation: 4
Reward: 0.0
False False
Action: 3
Observation: 4
Reward: 0.0
False False
Action: 0
Observation: 0
Reward: 0.0
False False
Action: 0
Observation: 4
Reward: 0.0
False False
Action: 0
Observation: 8
Reward: 0.0
False False
Action: 2
Observation: 4
Reward: 0.0
False False
Action: 3
Observation: 5
Reward: 0.0
False True


In [228]:
# make q_learning train function for 1 episode
def q_learning_episode(env, q_table):
    epsilon = 0.1
    alpha = 0.1
    gamma = 0.9
    done = False
    steps = 0
    state = env.reset(seed=42)
    while not done:
        # POLICY
        if type(state)==int:    # state troca entre 0 e (0,prob=1), não sei porque, dai todos os if, else
            previous_state=state
        else:
            previous_state=state[0]
        if np.random.random() < epsilon:
            action = np.random.randint(0,4)
        else:
            if type(state)==int:
                action =np.argmax(q_table[state])
            else:
                action =np.argmax(q_table[state[0]])
        state, reward, done, _ ,_= env.step(action)
        reward-= 1
        if state in [5, 7, 11, 12]:
            reward -= 20
        elif state == 15:
            reward += 50
        next_state=state
        new_state_max = np.max(q_table[next_state])
        #q_table[state, action] = (1 - alpha)*q_table[state, action] + alpha*(reward + gamma*new_state_max - q_table[state, action])
        q_table[previous_state,action] += alpha*(reward + gamma*new_state_max - q_table[previous_state,action])
        # in case the episode is taking too many steps to finish
        steps+=1
        if steps >= 400:
            break        
    return (q_table)


In [229]:
import gym
import numpy as np
def fazer_q_table():
    env = gym.make('FrozenLake-v1',is_slippery=True)

    number_of_actions = env.action_space.n
    number_of_states = env.observation_space.n

# Initialize Q-Table
    q_table = np.array(np.zeros((number_of_states, number_of_actions)))
    print('Q Table - SHAPE:', q_table.shape)
    # number of episodes to train the agent
    n_episodes = 120
    for e in range(n_episodes):
        q_table = q_learning_episode(env, q_table)
    print(q_table)
    return q_table

In [230]:
# Show results of training

def evaluate_q_learning(env, q_table):
    def policy(q_table, observation):
        action = np.argmax(q_table[observation])# replace this with the right policy
        return action
    observation = env.reset(seed=42)
    observation=0
    done = False
    nstep=0
    while not done:
        action = policy(q_table, observation)
        observation, reward, done,_,_ = env.step(action)
        print('Action:', action)
        print('Observation:', observation)
        print('Reward:', reward)
        env.render()
        nstep+=1      # Contar número de passos
    print(f'Número de passos: {nstep}')
    env.close()
env = gym.make('FrozenLake-v1',render_mode="human",is_slippery=True)
observation = env.reset(seed=42)
q_table=fazer_q_table()             # refazer q table
# use q_table to evaluate function
evaluate_q_learning(env,q_table)

with open('sample.txt', 'w') as f:    # Guardar q table
    f.write(str(q_table))  

Q Table - SHAPE: (16, 4)
[[-2.50846591  4.27894192 -3.71142622 -2.37125588]
 [-9.66971259 -5.68731152 -4.30678344 -1.59522252]
 [-3.24968072 -3.45197884 -3.4730495  -3.42559341]
 [-5.58197547 -6.15861563 -4.17550059 -3.8821978 ]
 [10.87339061 -3.47494663 -3.87421463 -4.07641912]
 [ 0.          0.          0.          0.        ]
 [-5.66297833 -7.47295498 -7.77478929 -7.17848514]
 [ 0.          0.          0.          0.        ]
 [-5.7639      3.88669167 -2.84253713 18.5894936 ]
 [ 3.88366504 27.55252195 -2.19        2.37272171]
 [32.30712088  2.3676687  -3.28898747  4.2846791 ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 2.67252731 -0.19       24.81691368 -5.52662602]
 [ 6.78400104 45.9204229   9.08448195  1.97356105]
 [ 0.          0.          0.          0.        ]]
Action: 1
Observation: 4
Reward: 0.0
Action: 0
Observation: 8
Reward: 0.0
Action: 3
Observation: 8
Reward: 0.0
Action: 3
Observation: 9
Reward: 0.0
Action: 1
